In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs

In [ ]:
import os
import sys
from pathlib import Path
SCRIPT_DIR = os.path.dirname(os.path.abspath("__init__.py"))
SRC_DIR = Path(SCRIPT_DIR).parent.parent.absolute()
print(SCRIPT_DIR)
print(SRC_DIR)
sys.path.append(os.path.dirname(SRC_DIR))

In [ ]:
from src.experiments_evaluation.validation_helpers import scale_t2m_back, scale_slp_back

In [ ]:
MONTHS = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]
dates_val = pd.date_range(start="1955-01-01", periods=3644, freq="D")
dates_test = pd.date_range(start="1950-01-01", periods=1820, freq="D")

## Load errors

### Validation MCAR

In [ ]:
temp_ta_val_mcar = scale_t2m_back(np.load("../experiments_evaluation/ex5.1_elev_mov_cmi/errors/temp_t2m_err_99p.npy"), 
                         for_error=True)
temp_slp_val_mcar = scale_slp_back(np.load("../experiments_evaluation/ex5.1_elev_mov_cmi/errors/temp_slp_err_99p.npy"),
                              for_error=True)
print(temp_ta_val_mcar.shape)
print(temp_slp_val_mcar.shape)

spat_ta_val_mcar = scale_t2m_back(np.load("../experiments_evaluation/ex5.1_elev_mov_cmi/errors/spat_t2m_err_99p.npy"), 
                         for_error=True)
spat_slp_val_mcar = scale_slp_back(np.load("../experiments_evaluation/ex5.1_elev_mov_cmi/errors/spat_slp_err_99p.npy"),
                              for_error=True)
print(spat_ta_val_mcar.shape)
print(spat_slp_val_mcar.shape)

## Validation NMAR

In [ ]:
temp_ta_val_nmar = scale_t2m_back(np.load("../experiments_evaluation/ex7.2_emc_mnar_training/errors/temp_t2m_err_99p.npy"), 
                         for_error=True)
temp_slp_val_nmar = scale_slp_back(np.load("../experiments_evaluation/ex7.2_emc_mnar_training/errors/temp_slp_err_99p.npy"),
                              for_error=True)
print(temp_ta_val_nmar.shape)
print(temp_slp_val_nmar.shape)

spat_ta_val_nmar = scale_t2m_back(np.load("../experiments_evaluation/ex7.2_emc_mnar_training/errors/spat_t2m_err_99p.npy"), 
                         for_error=True)
spat_slp_val_nmar = scale_slp_back(np.load("../experiments_evaluation/ex7.2_emc_mnar_training/errors/spat_slp_err_99p.npy"),
                              for_error=True)
print(spat_ta_val_nmar.shape)
print(spat_slp_val_nmar.shape)

## Test

In [ ]:
temp_ta_test = scale_t2m_back(np.load("outputs/temp_t2m_err_99p.npy"), 
                         for_error=True)
temp_slp_test = scale_slp_back(np.load("outputs/temp_slp_err_99p.npy"),
                              for_error=True)
print(temp_ta_test.shape)
print(temp_slp_test.shape)

spat_ta_test = scale_t2m_back(np.load("outputs/spat_t2m_err_99p.npy"), 
                         for_error=True)
spat_slp_test = scale_slp_back(np.load("outputs/spat_slp_err_99p.npy"),
                              for_error=True)
print(spat_ta_test.shape)
print(spat_slp_test.shape)

# Maps (Spatial)

In [ ]:
# Define reduced size window
from src.config import BASE_LAT_START, BASE_LAT_END, BASE_LON_START, BASE_LON_END

In [ ]:
def round_to_nearest_step(matrix, step=0.2):
    return np.round(matrix / step) * step

In [ ]:
ta_spat_min = np.min([np.min(spat_ta_val_mcar), np.min(spat_ta_val_nmar), np.min(spat_ta_test)])
ta_spat_max = np.max([np.max(spat_ta_val_mcar), np.max(spat_ta_val_nmar), np.max(spat_ta_test)])
ta_step = 0.2
ta_boundaries = np.arange(ta_spat_min, ta_spat_max + ta_step, ta_step)
ta_ticks = np.arange(ta_spat_min, ta_spat_max + ta_step, ta_step)

slp_spat_min = np.min([np.min(spat_slp_val_mcar), np.min(spat_slp_val_nmar), np.min(spat_slp_test)])
slp_spat_max = np.max([np.max(spat_slp_val_mcar), np.max(spat_slp_val_nmar), np.max(spat_slp_test)])
slp_step = 100
slp_boundaries = round_to_nearest_step(np.arange(slp_spat_min, slp_spat_max + slp_step, slp_step), slp_step)
slp_ticks = round_to_nearest_step(np.arange(slp_spat_min, slp_spat_max + slp_step, slp_step), slp_step)

print(slp_spat_min, slp_spat_max)

extent = [BASE_LON_START, BASE_LON_END, BASE_LAT_START,  BASE_LAT_END]

col_bar_width = 0.02
col_bar_x_offset = 0.005

fig = plt.figure(figsize=(20, 15))

# SPATIAL
# Validation Temperature MCAR
ax00 = fig.add_subplot(3, 2, 1, projection=ccrs.PlateCarree())
ax00.coastlines(resolution='110m')
ax00.set_extent(extent, ccrs.PlateCarree())
im00 = ax00.imshow(round_to_nearest_step(spat_ta_val_mcar, ta_step), 
                 transform=ccrs.PlateCarree(),
                 extent=extent,
                 origin='lower',
                 interpolation='nearest',
                 cmap='Reds',
                   vmin=ta_spat_min, vmax=ta_spat_max,
                   )
cax00 = fig.add_axes([ax00.get_position().x1 + col_bar_x_offset, 
                      ax00.get_position().y0, 
                      col_bar_width, 
                      ax00.get_position().height])
# Create discrete boundaries and ticks for the colorbar
plt.colorbar(im00, cax=cax00, boundaries=ta_boundaries, ticks=ta_ticks)
ax00.set_title(f"Validation Temperature MCAR MAE [°C]", fontsize=16)

# Validation Pressure MCAR
ax01 = fig.add_subplot(3, 2, 2, projection=ccrs.PlateCarree())
ax01.coastlines(resolution='110m')
ax01.set_extent(extent, ccrs.PlateCarree())#,
im01 = ax01.imshow(round_to_nearest_step(spat_slp_val_mcar, slp_step), 
                 transform=ccrs.PlateCarree(),
                 extent=extent,
                 origin='lower',
                 interpolation='nearest',
                 cmap='Reds',
                   vmin=slp_spat_min, vmax=slp_spat_max,)

cax01 = fig.add_axes([ax01.get_position().x1 + col_bar_x_offset, 
                      ax01.get_position().y0, 
                      col_bar_width,
                      ax01.get_position().height])
plt.colorbar(im01, cax=cax01, boundaries=slp_boundaries, ticks=slp_ticks)
ax01.set_title(f"Validation Pressure MCAR MAE [Pa]", fontsize=16)


# Validation Temperature NMAR
ax10 = fig.add_subplot(3, 2, 3, projection=ccrs.PlateCarree())
ax10.coastlines(resolution='110m')
ax10.set_extent(extent, ccrs.PlateCarree())
im10 = ax10.imshow(round_to_nearest_step(spat_ta_val_nmar, ta_step), 
                 transform=ccrs.PlateCarree(),
                 extent=extent,
                 origin='lower',
                 interpolation='nearest',
                 cmap='Reds',
                   vmin=ta_spat_min, vmax=ta_spat_max,
                   )
cax10 = fig.add_axes([ax10.get_position().x1 + col_bar_x_offset, 
                      ax10.get_position().y0, 
                      col_bar_width, 
                      ax10.get_position().height])
plt.colorbar(im10, cax=cax10, boundaries=ta_boundaries, ticks=ta_ticks)
ax10.set_title(f"Validation Temperature HiMi MAE [°C]", fontsize=16)  # HiMi = NMAR

# Validation Pressure NMAR
ax11 = fig.add_subplot(3, 2, 4, projection=ccrs.PlateCarree())
ax11.coastlines(resolution='110m')
ax11.set_extent(extent, ccrs.PlateCarree())#,
im11 = ax11.imshow(round_to_nearest_step(spat_slp_val_nmar, slp_step), 
                 transform=ccrs.PlateCarree(),
                 extent=extent,
                 origin='lower',
                 interpolation='nearest',
                 cmap='Reds',
                   vmin=slp_spat_min, vmax=slp_spat_max,)

cax11 = fig.add_axes([ax11.get_position().x1 + col_bar_x_offset, 
                      ax11.get_position().y0, 
                      col_bar_width,
                      ax11.get_position().height])
plt.colorbar(im11, cax=cax11, boundaries=slp_boundaries, ticks=slp_ticks)
ax11.set_title(f"Validation Pressure HiMi MAE [Pa]", fontsize=16)  # HiMi = NMAR




# Test Temperature
ax20 = fig.add_subplot(3, 2, 5, projection=ccrs.PlateCarree())
ax20.coastlines(resolution='110m')
ax20.set_extent(extent, ccrs.PlateCarree())
im20 = ax20.imshow(round_to_nearest_step(spat_ta_test, ta_step), 
                 transform=ccrs.PlateCarree(),
                 extent=extent,
                 origin='lower',
                 interpolation='nearest',
                 cmap='Reds',
                   vmin=ta_spat_min, vmax=ta_spat_max,)
cax20 = fig.add_axes([ax20.get_position().x1 + col_bar_x_offset, 
                      ax20.get_position().y0,
                      col_bar_width, 
                      ax20.get_position().height])
plt.colorbar(im20, cax=cax20, boundaries=ta_boundaries, ticks=ta_ticks)
ax20.set_title(f"Test Temperature MAE [°C]", fontsize=16)


# Test Pressure
ax21 = fig.add_subplot(3, 2, 6, projection=ccrs.PlateCarree())
ax21.coastlines(resolution='110m')
ax21.set_extent(extent, ccrs.PlateCarree())
im21 = ax21.imshow(round_to_nearest_step(spat_slp_test, slp_step), 
                 transform=ccrs.PlateCarree(),
                 extent=extent,
                 origin='lower',
                 interpolation='nearest',
                 cmap='Reds',
                   vmin=slp_spat_min, vmax=slp_spat_max,)
cax21 = fig.add_axes([ax21.get_position().x1 + col_bar_x_offset, 
                      ax21.get_position().y0,
                      col_bar_width, 
                      ax21.get_position().height])
plt.colorbar(im21, cax=cax21, boundaries=slp_boundaries, ticks=slp_ticks)
ax21.set_title(f"Test Pressure MAE [Pa]", fontsize=16)


plt.savefig(f"figures/val_test_spatial_v2.png", 
            bbox_inches='tight',
            pad_inches=0.1)

# Boxplots (Temporal)

In [ ]:
df_temporal_val_mcar = pd.DataFrame({
    "err_ta": temp_ta_val_mcar,
    "err_slp": temp_slp_val_mcar,
    "month": dates_val.month
})
df_temporal_val_nmar = pd.DataFrame({
    "err_ta": temp_ta_val_nmar,
    "err_slp": temp_slp_val_nmar,
    "month": dates_val.month
})
df_temporal_test = pd.DataFrame({
    "err_ta": temp_ta_test,
    "err_slp": temp_slp_test,
    "month": dates_test.month
})

In [ ]:
fig = plt.figure(figsize=(16, 12))

# BOX PLOTS

# Validation Temperature MCAR
ax00 = fig.add_subplot(3, 2, 1)
df_temporal_val_mcar.boxplot(column=['err_ta'], by='month', ax=ax00)
ax00.set(xlabel="", ylabel='')
ax00.set_xticks([m+1 for m in range(12)], [MONTHS[i] for i in range(12)])
ax00.set_title(f"Validation Temperature MCAR MAE [°C]", fontsize=16)

# Validation Pressure MCAR
ax01 = fig.add_subplot(3, 2, 2)
df_temporal_val_mcar.boxplot(column=['err_slp'], by='month', ax=ax01)
ax01.set(xlabel="", ylabel='')
ax01.set_xticks([m+1 for m in range(12)], [MONTHS[i] for i in range(12)])
ax01.set_title(f"Validation Pressure MCAR MAE [Pa]", fontsize=16)

# Validation Temperature NMAR
ax10 = fig.add_subplot(3, 2, 3)
df_temporal_val_nmar.boxplot(column=['err_ta'], by='month', ax=ax10)
ax10.set(xlabel="", ylabel='')
ax10.set_xticks([m+1 for m in range(12)], [MONTHS[i] for i in range(12)])
ax10.set_title(f"Validation Temperature HiMi MAE [°C]", fontsize=16)  # HiMi = NMAR

# Validation Pressure NMAR
ax11 = fig.add_subplot(3, 2, 4)
df_temporal_val_nmar.boxplot(column=['err_slp'], by='month', ax=ax11)
ax11.set(xlabel="", ylabel='')
ax11.set_xticks([m+1 for m in range(12)], [MONTHS[i] for i in range(12)])
ax11.set_title(f"Validation Pressure HiMi MAE [Pa]", fontsize=16)  # HiMi = NMAR


# TEST Temperature
ax20 = fig.add_subplot(3, 2, 5)
df_temporal_test.boxplot(column=['err_ta'], by='month', ax=ax20)
ax20.set(xlabel="", ylabel='')
ax20.set_xticks([m+1 for m in range(12)], [MONTHS[i] for i in range(12)])
ax20.set_title(f"Test Temperature MAE [°C]", fontsize=16)

# Test Pressure
ax21 = fig.add_subplot(3, 2, 6)
df_temporal_test.boxplot(column=['err_slp'], by='month', ax=ax21)
ax21.set(xlabel="", ylabel='')
ax21.set_xticks([m+1 for m in range(12)], [MONTHS[i] for i in range(12)])
ax21.set_title(f"Test Pressure MAE [Pa]", fontsize=16)

plt.suptitle('')

plt.savefig(f"figures/val_test_temporal_v2.png", 
            bbox_inches='tight',
            pad_inches=0.1)